In [1]:
import numpy as np
import pandas as pd

from src.reader import Reader
from src.downloader import Downloader
from src.localreader import LocalReader, SHRINK, CENTERING, getMean

In [2]:
#Download images Starting from 1st challange to 10th challange
#In other words dowload 10 1st subsets
#downloader = Downloader(start=0, end=10)
#downloader.download(path="data/AVA.txt")

In [6]:
reader = LocalReader(width=256, height=256, shaping=SHRINK)
for images, scores in reader.read(start=0, end=3):
    print(images.shape)
    print(scores.shape)
    score_mean = getMean(scores)
    print(score_mean.shape)
    print(pd.Series(score_mean).describe())

[============================================================] 100.0% .531.tar.gz    
(234, 256, 256, 3)
(234, 10)
(234,)
count    234.000000
mean       5.130067
std        0.626367
min        2.812227
25%        4.711046
50%        5.166984
75%        5.531546
max        6.995763
dtype: float64
[============================================================] 100.0% .430.tar.gz    
(828, 256, 256, 3)
(828, 10)
(828,)
count    828.000000
mean       5.635811
std        0.690438
min        3.196305
25%        5.174244
50%        5.627838
75%        6.119288
max        7.987448
dtype: float64
[============================================================] 100.0% .859.tar.gz    
(194, 256, 256, 3)
(194, 10)
(194,)
count    194.000000
mean       5.764495
std        0.633411
min        3.937063
25%        5.358699
50%        5.804905
75%        6.171088
max        7.223776
dtype: float64
